# Comparison with package ```BNPmix```

In [ ]:
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from google.protobuf.internal.decoder import _DecodeVarint32
import sys
sys.path.insert(0, '..')
from proto.py.algorithm_state_pb2 import AlgorithmState
import arviz as az

# Define paths
CSVIN =  '../resources/csv/in/thesis'
CSVOUT = '../resources/csv/out/thesis'
RECORDIO = '../resources/recordio'
# Utility to read file collector, courtesy of
# github.com/mberaha/utils/blob/master/proto_utils/py/recordio.py
def readManyFromFile(filename, msgType):
    out = []
    with open(filename, "rb") as fp:
        buf = fp.read()
    n = 0
    while n < len(buf):
        msg_len, new_pos = _DecodeVarint32(buf, n)
        n = new_pos
        msg_buf = buf[n:n+msg_len]
        try:
            msg = msgType()
            msg.ParseFromString(msg_buf)
            out.append(msg)
            n += msg_len
        except Exception as e:
            break
    return out

## ```galaxy``` dataset

In [ ]:
# Read data and grid
galaxy_data = np.genfromtxt(f"{CSVIN}/galaxy.csv")
galaxy_grid = np.genfromtxt(f"{CSVIN}/galaxy_grid.csv")

In [ ]:
# Define algorithms and mixings lists
mix = "PY"
algos1 = 'Neal2 Neal3 Neal8'.split()
algos2 = 'mar ics'.split()

In [ ]:
# Read densities
galaxy_dens_1 = dict()
galaxy_dens_2 = dict()
for algo in algos1:
    fdens = f"{CSVOUT}/galaxy_dens_{algo}_{mix}.csv"
    galaxy_dens_1[algo] = np.genfromtxt(fdens, delimiter=',')
for algo in algos2:
    fdens = f"{CSVOUT}/galaxy_dens_bnpmix_{algo}.csv"
    galaxy_dens_2[algo] = np.genfromtxt(fdens, delimiter=',')

In [ ]:
# Plot densities
fig = plt.figure(figsize=(10,6))
plt.hist(galaxy_data, density=True, color='lightgray')
# Means use 1-in-2 thinning
for algo in algos1:
    dens = np.exp(np.mean(galaxy_dens_1[algo][0::2], axis=0))
    plt.plot(galaxy_grid, dens)
for algo in algos2:
    dens = np.mean(galaxy_dens_2[algo][0::2], axis=0)
    plt.plot(galaxy_grid, dens, linestyle='--')
plt.title("Pitman-Yor density estimation with bayesmix (solid) and bnpmix (dashed)")
plt.legend(algos1 + algos2)

## ```faithful``` dataset

In [ ]:
# Read data and grid
faithful_data = np.genfromtxt(f"{CSVIN}/faithful.csv")
faithful_grid = np.genfromtxt(f"{CSVIN}/faithful_grid.csv")

In [ ]:
# Define algorithms and mixings lists
mix = "PY"
algos1 = 'Neal2 Neal3 Neal8'.split()
algos2 = 'mar ics'.split()

In [ ]:
# Read densities
faithful_dens_1 = dict()
faithful_dens_2 = dict()
for algo in algos1:
    fdens = f"{CSVOUT}/faithful_dens_{algo}_{mix}.csv"
    faithful_dens_1[algo] = np.genfromtxt(fdens, delimiter=',')
for algo in algos2:
    fdens = f"{CSVOUT}/faithful_dens_bnpmix_{algo}.csv"
    faithful_dens_2[algo] = np.genfromtxt(fdens, delimiter=',')

In [ ]:
# Plot densities (contours in log-scale)
fig = plt.figure(figsize=(14,18))
fig.suptitle(7*" " + "Pitman-Yor log-density estimation", y=0.92, size='xx-large')
algos_all = 'Neal8 mar Neal2 ics'.split()
idx = 0
for idx in range(1, len(algos_all)+1):
    algo = algos_all[idx-1]
    # Means use 1-in-2 thinning
    if idx % 2 == 1:
        # bayesmix
        dens = np.mean(faithful_dens_1[algo][0::2], axis=0).reshape(-1,1)
        lib = "bayesmix"
    else:
        # BNPmix
        dens = np.log(np.mean(faithful_dens_2[algo][0::2], axis=0)).reshape(-1,1)
        lib = "BNPmix"
    plot_data = pd.DataFrame(np.hstack([faithful_grid, dens]), 
                             columns=["x", "y", "z"])
    Z = plot_data.pivot_table(index='x', columns='y', values='z').T.values
    X_unique = np.sort(plot_data.x.unique())
    Y_unique = np.sort(plot_data.y.unique())
    X, Y = np.meshgrid(X_unique, Y_unique)
    ax = fig.add_subplot(3, 2, idx)
    if Z.any():
        ax.contour(X, Y, Z)
    ax.set_title(f"{lib}'s {algo}")